# Micro Passes
> passes over the AST of a statement to do semantic checks and register state in the session object

In [ ]:
#| default_exp micro_passes

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from abc import ABC, abstractmethod
import pytest

# from lark import Transformer, Token
# from lark import Tree as LarkNode
# from lark.visitors import Interpreter, Visitor_Recursive, Visitor
from pathlib import Path
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
from pydantic import BaseModel
import networkx as nx

# from spannerlib.ast_node_types import (Assignment, ReadAssignment, AddFact, RemoveFact, Query, Rule, IERelation, RelationDeclaration, Relation)
from spannerlib.primitive_types import Span, DataTypes, DataTypeMapping
# from spannerlib.engine import RESERVED_RELATION_PREFIX
# # from spannerlib.graphs import NetxStateGraph
# from spannerlib.symbol_table import SymbolTableBase
# from spannerlib.general_utils import (get_free_var_names, get_output_free_var_names, get_input_free_var_names, fixed_point, check_properly_typed_relation, type_check_rule_free_vars)
# from spannerlib.passes_utils import assert_expected_node_structure, unravel_lark_node,ParseNodeType

from graph_rewrite import draw, draw_match, rewrite, rewrite_iter

from spannerlib.grammar import parse_spannerlog

## Scaffolding

In [ ]:
def serialize_tree(g):
    root = next(nx.topological_sort(g))
    return nx.tree_data(g,root) 


In [ ]:
class DummySession():
    def __init__(self,passes=None):
        if passes is None:
            passes = []
        self.passes = passes
        # self.term_graph = nx.DiGraph()
        # self.symbol_table = dict() # {var_name:(type,value)}
        # self.ie_functions = dict() # {ie_func_name: {
        #                             #     'in_schema': ...,
        #                             #     'out_schema': ...,
        #                             #     'func': function,
        #                             #     'func_type': ... # per row, bulk, aggregation in the future etc
        #                             #     }
        #                             # }
        # #self.db # db entry points # TODO maybe this shouldnt belong here

    def run_query(self, query):
        statements = parse_spannerlog(query,split_statements=True)
        clean_asts = []
        for statement in statements:
            ast = statement
            for pass_ in self.passes:
                pass_(ast,self)
            clean_asts.append(ast)
        
        return clean_asts
    

In [ ]:
# TODO from here make uniq node names start from 0 every time
# Todo make a util to compare nx to json, not tree and change grammar to use it.
sess = DummySession()

#TODO from here, figure out why 'decl_string' has a val string while being a leaf

asts = sess.run_query("""
            new string(str)
            string("a")
            string_length(Str, Len) <- string(Str), Length(Str) -> (Len)
            """)
for ast in asts:
    draw(ast)

## Data Types

In [ ]:
#| export
from enum import Enum
from typing import Any
from pydantic import ConfigDict

class Span(BaseModel):
    start: int
    end: int

    def __lt__(self, other: Span) -> bool:
        if self.start == other.start:
            return self.end < other.end

        return self.start < other.start

    # # used for sorting `Span`s in dataframes
    # def __hash__(self) -> int:
    #     return hash((self.start, self.end))

class Var(BaseModel):
    name: str

class FreeVar(BaseModel):
    name: str

PrimitiveType=Union[str,int,Span]
Type = Union[PrimitiveType,Var,FreeVar]

class RelationDefinition(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    scheme: List[type]

class Relation(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    terms: List[Type]

class IEFunction(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    in_schema: List[type]
    out_schema: List[type]
    func: Callable

class IERelation(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    in_terms: List[Type]
    out_terms: List[Type]

class Rule(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    head: Relation
    body: List[Union[Relation,IERelation]]

In [ ]:
RelationDefinition(name='x',scheme=[str,int,Span])

RelationDefinition(name='x', scheme=[<class 'str'>, <class 'int'>, <class '__main__.Span'>])

# Semantic Checks

## convert_primitive_values_to_objects

In [ ]:
def convert_primitive_values_to_objects(ast,session):

    # primitive values
    def cast_new_value(match):
        val_type = match['var']['type']
        value = match['val_node']['val']
        if val_type == 'integer':
            value = int(value)
        elif val_type == 'var_name':
            value = Var(name=value)
        elif val_type == 'free_var_name':
            value = FreeVar(name=value)
        else:#str
            value = str(value)
        return value
    
    rewrite(ast,
        lhs='var[type]->val_node[val]',
        p='var[type]',
        rhs='var[type,val={{new_val}}]',
        condition= lambda match: match['var']['type'] in ['string','integer','var_name','relation_name','free_var_name'],
        render_rhs={'new_val': cast_new_value},
        # display_matches=True
        )

    # span object from 2 integers
    for match in rewrite_iter(ast,
        lhs='u[type="span"]-[idx=0]->v;u-[idx=1]->w',
        p='u[type]'):
        match['u']['val']=Span(start=match['v']['val'],end=match['w']['val'])

    # schema types into class types
    decl_type_to_class = {
        'decl_string':str,
        'decl_int':int,
        'decl_span':Span,
    }

    for decl_type,decl_class in decl_type_to_class.items():
        for match in rewrite_iter(ast,lhs=f'x[val="{decl_type}"]'):
            match['x']['val']=decl_class


In [ ]:
sess = DummySession(passes=[
  # convert_primitive_values_to_objects
  ])
asts = sess.run_query("""
            x="a"
            """)
for ast in asts:
    draw(ast)



In [ ]:
sess = DummySession(passes=[convert_primitive_values_to_objects])
asts = sess.run_query("""
            x=1
            S("a",1,[4,5))
            new R(int,str)
            ?R(x,X)
            R(X,Y)<-S(X,Y)
            """)
for ast in asts:
    draw(ast)

assert ([serialize_tree(ast) for ast in asts] ==  [{'type': 'assignment',
  'id': 0,
  'children': [{'type': 'var_name', 'val': Var(name='x'), 'id': 1},
   {'type': 'integer', 'val': 1, 'id': 3}]},
 {'type': 'add_fact',
  'id': 0,
  'children': [{'type': 'relation_name', 'val': 'S', 'id': 1},
   {'type': 'const_term_list',
    'id': 3,
    'children': [{'type': 'string', 'val': '"a"', 'id': 4},
     {'type': 'integer', 'val': 1, 'id': 6},
     {'type': 'span', 'val': Span(start=4, end=5), 'id': 8}]}]},
 {'type': 'relation_declaration',
  'id': 0,
  'children': [{'type': 'relation_name', 'val': 'R', 'id': 1},
   {'type': 'decl_term_list',
    'id': 3,
    'children': [{'val': int, 'id': 4}, {'val': str, 'id': 5}]}]},
 {'type': 'query',
  'id': 0,
  'children': [{'type': 'relation_name', 'val': 'R', 'id': 1},
   {'type': 'term_list',
    'id': 3,
    'children': [{'type': 'var_name', 'val': Var(name='x'), 'id': 4},
     {'type': 'free_var_name', 'val': FreeVar(name='X'), 'id': 6}]}]},
 {'type': 'rule',
  'id': 0,
  'children': [{'type': 'rule_head',
    'id': 1,
    'children': [{'type': 'relation_name', 'val': 'R', 'id': 2},
     {'type': 'free_var_name_list',
      'id': 4,
      'children': [{'type': 'free_var_name',
        'val': FreeVar(name='X'),
        'id': 5},
       {'type': 'free_var_name', 'val': FreeVar(name='Y'), 'id': 7}]}]},
   {'type': 'rule_body_relation_list',
    'id': 9,
    'children': [{'type': 'relation',
      'id': 10,
      'children': [{'type': 'relation_name', 'val': 'S', 'id': 11},
       {'type': 'term_list',
        'id': 13,
        'children': [{'type': 'free_var_name',
          'val': FreeVar(name='X'),
          'id': 14},
         {'type': 'free_var_name', 'val': FreeVar(name='Y'), 'id': 16}]}]}]}]}] )

## Remove newlines from strings

In [ ]:
#| export
def remove_new_lines_from_strings(ast,sess):
    for match in rewrite_iter(ast,
        lhs='v[type="string",val]'):
        # TODO we also remove the starting and ending quotes, TODO make them disapear in the parsing stage
        match['v']['val'] = match['v']['val'].replace('\\\n','')[1:-1]


In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    ])
asts = sess.run_query("""
x="hello \
world"
""")
for ast in asts:
    draw(ast)

ast = asts[0]
assert serialize_tree(ast) == {'type': 'assignment',
 'id': 0,
 'children': [{'type': 'var_name', 'val': Var(name='x'), 'id': 1},
  {'type': 'string', 'val': 'hello world', 'id': 3}]}

## Check reserved relation names

In [ ]:
class CheckReservedRelationNames():
    def __init__(self,reserved_prefix):
        self.reserved_prefix = reserved_prefix
    def __call__(self,ast,sess):
        for match in rewrite_iter(ast,lhs='X[type="relation_name",val]'):
            relation_name = match['X']['val']
            if relation_name.startswith(self.reserved_prefix):
                raise ValueError(f"Relation name '{relation_name}' starts with reserved prefix '{self.reserved_prefix}'")

In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    ])
asts = sess.run_query("""
            S("a",1)
            R(X,Y)<-S(X,Y),T(X,Y)
            """)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query("""
            spanner_S("a",1)
            """)
print(exc_info.value)


Relation name 'spanner_S' starts with reserved prefix 'spanner_'


In [ ]:
draw(asts[0])

## Check read assignments got existing path

In [ ]:
import os
def check_referenced_paths_exist(ast,sess):
    for match in rewrite_iter(ast,
    lhs='X[type="read_assignment"]-[idx=1]->PathNode[val]',
    # display_matches=True
    ):
        path = Path(match['PathNode']['val'])
        if not path.exists():
            raise ValueError(f'path {path} was not found in {os.getcwd()}')


In [ ]:
# check that read assignments got a string which is an existing path
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    check_referenced_paths_exist,
    ])

file = Path("file.txt")
file.touch()

# TODO figure out why this doesnt work
asts = sess.run_query(f"""
            x=read("file.txt")
            """)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query("""
            x=read("not_existing_file.txt")
            """)
print(exc_info.value)

file.unlink()

path not_existing_file.txt was not found in /Users/dean/tdk/spannerlib/nbs


## check reference vars are defined

In [ ]:
#| export
def check_referenced_vars_exist(ast,sess):

    # first rename all left hand sign variables 
    # as type "var_name_lhs"
    # so we can seperate them from reference variables
    for assignment_type in ["assignment","read_assignment"]:
        for match in rewrite_iter(ast,
                lhs=f"""X[type="{assignment_type}"]-[idx=0]->LHS[type="var_name",val]"""
                ):
            match['LHS']['type'] = "var_name_lhs"

    # now for each reference variable check if it is in the symbol table
    for match in rewrite_iter(ast,lhs=f"""X[type="var_name",val]"""):
        var_name = match['X']['val'].name
        if not var_name in sess.symbol_table:
            raise ValueError(f'Variable {var_name} is not defined')


In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    check_referenced_paths_exist,
    check_referenced_vars_exist,
    ])

sess.symbol_table={'y':(int,1),'x':(str,"hello")}

asts = sess.run_query(f"""
            z=1
            x=y
            R(x,y)
            """)
with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                R(x,z)
                """)
print(exc_info.value)

# for ast in asts:
#     draw(ast)


Variable z is not defined


## Cast relations to python objects

In [ ]:
def relations_to_dataclasses(ast,sess):

   # regular relations
   #TODO another example where i need to edit the graph imperatively because i dont have horizontal recursion in LHS
   for match in rewrite_iter(ast,
      lhs='''
         statement[type]->name[type="relation_name",val];
         statement->terms[type]
         ''',
         #TODO i expect to be able to put an rhs here only, and if a p is not given, assume it is the identity over nodes in LHS
         p='statement[type]',
         condition=lambda match: (match['statement']['type'] in ['add_fact','remove_fact','relation','rule_head','query']
                                   and match['terms']['type'] in ['const_term_list','term_list','free_var_name_list'])
         ):
      term_nodes = list(ast.successors(match.mapping['terms']))
      #TODO check we iterate in order on the children
      match['statement']['val'] = Relation(name=match['name']['val'],terms=[ast.nodes[term_node]['val'] for term_node in term_nodes])
      ast.remove_nodes_from(term_nodes)
   # relation declerations
   for match in rewrite_iter(ast,
      lhs='''
         statement[type="relation_declaration"]->name[type="relation_name",val];
         statement->terms[type="decl_term_list"]
         ''',
         p='statement[type]'):
      term_nodes = list(ast.successors(match.mapping['terms']))
      match['statement']['val'] = RelationDefinition(name=match['name']['val'],scheme=[ast.nodes[term_node]['val'] for term_node in term_nodes])
      ast.remove_nodes_from(term_nodes)

   # ie relations
   for match in rewrite_iter(ast,
      lhs='''
         statement[type="ie_relation"]->name[type="relation_name",val];
         statement-[idx=1]->in_terms[type="term_list"];
         statement-[idx=2]->out_terms[type="term_list"]
      ''',p='statement[type]'):
      in_term_nodes = list(ast.successors(match.mapping['in_terms']))
      out_term_nodes = list(ast.successors(match.mapping['out_terms']))

      match['statement']['val'] = IERelation(name=match['name']['val'],
                                             in_terms=[ast.nodes[term_node]['val'] for term_node in in_term_nodes],
                                             out_terms=[ast.nodes[term_node]['val'] for term_node in out_term_nodes]
                                             )
      ast.remove_nodes_from(in_term_nodes+out_term_nodes)

In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    check_referenced_paths_exist,
    check_referenced_vars_exist,
    relations_to_dataclasses
    ])
asts = sess.run_query("""
R("hello",6)
R("hello",[4,5))<-True
R("hello",[4,5))<-False
new R(str,span,int,int)
?R("hello",Y)
R(X,Y)<-S(X,Y),T(X,Y)->(Y,Z)
""")
for ast in asts:
    draw(ast)

assert  [serialize_tree(ast) for ast in asts] == [{'type': 'add_fact',
  'val': Relation(name='R', terms=['hello', 6]),
  'id': 0,
  'children': []},
 {'type': 'add_fact',
  'val': Relation(name='R', terms=['hello', Span(start=4, end=5)]),
  'id': 0,
  'children': []},
 {'type': 'remove_fact',
  'val': Relation(name='R', terms=['hello', Span(start=4, end=5)]),
  'id': 0,
  'children': []},
 {'type': 'relation_declaration',
  'val': RelationDefinition(name='R', scheme=[str,Span,int,int]),
  'id': 0,
  'children': []},
 {'type': 'query',
  'val': Relation(name='R', terms=['hello', FreeVar(name='Y')]),
  'id': 0,
  'children': []},
 {'type': 'rule',
  'id': 0,
  'children': [{'type': 'rule_head',
    'val': Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
    'id': 1},
   {'type': 'rule_body_relation_list',
    'id': 9,
    'children': [{'type': 'relation',
      'val': Relation(name='S', terms=[FreeVar(name='X'), FreeVar(name='Y')]),
      'id': 10},
     {'type': 'ie_relation',
      'val': IERelation(name='T', in_terms=[FreeVar(name='X'), FreeVar(name='Y')], out_terms=[FreeVar(name='Y'), FreeVar(name='Z')]),
      'id': 18}]}]}]

## Relation referencing

* check that referenced relations and ie relations:
  * exist in the symbol table 
  * are called with the correct arity
  * are called with correct constants or vars types

In [ ]:
def verify_referenced_relations(ast,sess):

    def schema_match(types,vals):
        for type_,val in zip(types,vals):
            if isinstance(val,FreeVar):
                continue # free vars can be anything
            elif isinstance(val,Var):
                var_type = sess.symbol_table[val.name][0]
                if not type_ == var_type:
                    return False
            elif not isinstance(val,type_):
                return False
        return True

    for match in rewrite_iter(ast,
            lhs='''rel[type]''',
            condition=lambda match: match['rel']['type'] in ['add_fact','remove_fact','relation','query'],
            ):
        rel:Relation = match['rel']['val']
        if not rel.name in sess.relation_schemas:
            raise ValueError(f"Relation '{rel.name}' is not defined")
        expected_len = len(sess.relation_schemas[rel.name].scheme)
        if len(rel.terms) != expected_len:
            raise ValueError(f"Relation '{rel.name}' was called with {len(rel.terms)} terms but it was defined with {expected_len} terms")
        if not schema_match(sess.relation_schemas[rel.name].scheme,rel.terms):
            raise ValueError(f"Relation '{rel.name}' expected schema {sess.relation_schemas[rel.name].scheme} but got called with {rel.terms}")
      


In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    check_referenced_paths_exist,
    check_referenced_vars_exist,
    relations_to_dataclasses,
    verify_referenced_relations
    ])

sess.symbol_table ={
    'a':(int,1),
    'b':(str,"hello"),
}

sess.relation_schemas = {
    'R':RelationDefinition(name='R',scheme=[str,int]),
    'S':RelationDefinition(name='S',scheme=[str,int,int]),
                         }

sess.ie_funcs = {
    'T':IEFunction(name='T',in_schema=[str,int],out_schema=[int,str],func=lambda x,y:(y,x))
}


asts = sess.run_query("""
R("hello",6)
R("hello",a)
?R("hello",Y)
NewRel(X,Y)<-S(X,Y,3),T(X,Y)->(Y,Z)
""")

# for ast in asts:
#     draw(ast)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                Z("hello",4)
                """)
assert "Relation 'Z' is not defined" in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                R("hello",4,[4,5))
                """)
assert "Relation 'R' was called with 3 terms but it was defined with 2 terms" in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                R(4,4)
                """)
assert "Relation 'R' expected schema [<class 'str'>, <class 'int'>]" in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                R("hello",b)
                """)
assert "Relation 'R' expected schema [<class 'str'>, <class 'int'>]" in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    asts = sess.run_query(f"""
                ?R(4,Y)
                """)
assert "Relation 'R' expected schema [<class 'str'>, <class 'int'>]" in str(exc_info.value)
print(exc_info.value)
# assert  [serialize_tree(ast) for ast in asts] 
# [serialize_tree(ast) for ast in asts]

Relation 'Z' is not defined
Relation 'R' was called with 3 terms but it was defined with 2 terms
Relation 'R' expected schema [<class 'str'>, <class 'int'>] but got called with [4, 4]
Relation 'R' expected schema [<class 'str'>, <class 'int'>] but got called with ['hello', Var(name='b')]
Relation 'R' expected schema [<class 'str'>, <class 'int'>] but got called with [4, FreeVar(name='Y')]


## cast rules to data classes

In [ ]:

def rules_to_dataclasses(ast,sess):
   for match in rewrite_iter(ast,
      lhs='''
         statement[type="rule"]->head[type="rule_head",val];
         statement->body[type="rule_body_relation_list"]
      ''',p='statement[type]'):
      body_nodes = list(ast.successors(match.mapping['body']))
      head = match['head']['val']
      match['statement']['val'] = Rule(head=match['head']['val'],body=[ast.nodes[body_node]['val'] for body_node in body_nodes])
      ast.remove_nodes_from(body_nodes)
   return ast

In [ ]:
sess = DummySession(passes=[
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames('spanner_'),
    check_referenced_paths_exist,
    check_referenced_vars_exist,
    relations_to_dataclasses,
    # verify_referenced_relations,
    rules_to_dataclasses
    ])

asts = sess.run_query("""
R(X,Y,Z)<-S(X,Y),T(X,Y)
R(X,Y,Z)<-S(X,Y),T(X,Y)->(Y,Z)
""")
for ast in asts:
    draw(ast)

assert  [serialize_tree(ast) for ast in asts] == [{'type': 'rule',
  'val': Rule(head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]), body=[Relation(name='S', terms=[FreeVar(name='X'), FreeVar(name='Y')]), Relation(name='T', terms=[FreeVar(name='X'), FreeVar(name='Y')])]),
  'id': 0,
  'children': []},
 {'type': 'rule',
  'val': Rule(head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]), body=[Relation(name='S', terms=[FreeVar(name='X'), FreeVar(name='Y')]), IERelation(name='T', in_terms=[FreeVar(name='X'), FreeVar(name='Y')], out_terms=[FreeVar(name='Y'), FreeVar(name='Z')])]),
  'id': 0,
  'children': []}]

## Consistent Free Var types

In [ ]:
# TODO from here

In [ ]:
    """
    Performs semantic checks on rules using a Lark tree to ensure their safety. <br>
    A rule is considered "safe" when it meets certain conditions.


    * [Rule Safety Conditions](#rule-safety-conditions)
        * [Free Variable in Rule Head](#free-variable-in-rule-head)
        * [Bound Free Variable](#bound-free-variable)
    * [Examples](#examples)
    * [Safe Relations](#safe-relations)

    ---

    ### Rule Safety Conditions

    For a rule to be considered safe, the following two conditions must be met:

    ### 1. Free Variable in Rule Head

    Every free variable that appears in the rule head must occur at least once in the body as an output term of a relation.

    #### Examples

    * `parent(X,Y) <- son(X)` is not a safe rule because the free variable `Y` only appears in the rule head.  
    * `parent(X,Z) <- parent(X,Y), parent(Y,Z)` is a safe rule since both `X` and `Z` appear in the rule body.

    ### 2. Bound Free Variable

    A free variable is considered "bound" if it is constrained in a manner that limits the range of values it can take.

    To ensure that every free variable is bound, we must ensure that every relation in the rule body is a safe relation.

    ### Safe Relations

    A safe relation adheres to the following:

    * Its input relation is safe, meaning all its input's free variables are bound. Normal relations are always considered safe as they don't have input relations.  
    * A bound variable is one that exists in the output of a safe relation.

    #### Examples

    * `rel2(X,Y) <- rel1(X,Z), ie1<X>(Y)` is a safe rule as the only input free variable, `X`, exists in the output of the safe relation `rel1(X, Z)`.  
    * `rel2(Y) <- ie1<Z>(Y)` is not safe as the input free variable `Z` does not exist in the output of any safe relation.

    ---


## Rule safety

In [ ]:
#| export
class CheckRuleSafety(VisitorRecursivePass):
    """
    Performs semantic checks on rules using a Lark tree to ensure their safety. <br>
    A rule is considered "safe" when it meets certain conditions.


    * [Rule Safety Conditions](#rule-safety-conditions)
        * [Free Variable in Rule Head](#free-variable-in-rule-head)
        * [Bound Free Variable](#bound-free-variable)
    * [Examples](#examples)
    * [Safe Relations](#safe-relations)

    ---

    ### Rule Safety Conditions

    For a rule to be considered safe, the following two conditions must be met:

    ### 1. Free Variable in Rule Head

    Every free variable that appears in the rule head must occur at least once in the body as an output term of a relation.

    #### Examples

    * `parent(X,Y) <- son(X)` is not a safe rule because the free variable `Y` only appears in the rule head.  
    * `parent(X,Z) <- parent(X,Y), parent(Y,Z)` is a safe rule since both `X` and `Z` appear in the rule body.

    ### 2. Bound Free Variable

    A free variable is considered "bound" if it is constrained in a manner that limits the range of values it can take.

    To ensure that every free variable is bound, we must ensure that every relation in the rule body is a safe relation.

    ### Safe Relations

    A safe relation adheres to the following:

    * Its input relation is safe, meaning all its input's free variables are bound. Normal relations are always considered safe as they don't have input relations.  
    * A bound variable is one that exists in the output of a safe relation.

    #### Examples

    * `rel2(X,Y) <- rel1(X,Z), ie1<X>(Y)` is a safe rule as the only input free variable, `X`, exists in the output of the safe relation `rel1(X, Z)`.  
    * `rel2(Y) <- ie1<Z>(Y)` is not safe as the input free variable `Z` does not exist in the output of any safe relation.

    ---


    """
    pass


# Execution passes

In [ ]:
* Resolve Vars
* Register Vars
* Register new Relations
* Add/remove facts
* Add rules
* Run Queries

# Old

::: {.callout-note}
note that after using this pass, non statement nodes will no longer appear in the tree, so passes that
should work on said nodes need to be used before this pass in the passes pipeline (e.g. `FixString`).
:::

## Referenced vars

In [ ]:
# TODO move all micropasses that have to do with assignment here
#| export
class CheckDefinedReferencedVariables(InterpreterPass):
    """
    A lark tree semantic check. <br>
    checks whether each variable reference refers to a defined variable.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    def _assert_var_defined(self, var_name: str) -> None:
        """
        A utility function that checks if a variable is a defined variable in the symbol table.
        if not, raises an exception

        @param var_name: the name of the variable that will be checked
        """
        if not self.symbol_table.contains_variable(var_name):
            raise Exception(f'variable "{var_name}" is not defined')

    def _assert_var_terms_defined(self, term_list: Sequence[DataTypeMapping.term], type_list: Sequence[DataTypes]) -> None:
        """
        A utility function that checks if the non free variables in a term list are defined
        if one of them is not defined, raises an exception.

        @param term_list: a list of terms.
        @param type_list: the type of terms in term_list.
        """
        for term, term_type in zip(term_list, type_list):
            if term_type is DataTypes.var_name:
                # found a variable, check if it is defined
                assert isinstance(term, str), "a var_name must be of type str"
                self._assert_var_defined(term)

    @unravel_lark_node
    def assignment(self, assignment: Assignment) -> None:
        if assignment.value_type is DataTypes.var_name:
            # the assigned expression is a variable, check if it is defined
            assert isinstance(assignment.value, str), "a var_name must be of type str"
            self._assert_var_defined(assignment.value)

    @unravel_lark_node
    def read_assignment(self, assignment: ReadAssignment) -> None:
        if assignment.read_arg_type is DataTypes.var_name:
            # a variable is used as the argument for read(), check if it is defined
            assert isinstance(assignment.read_arg, str), "a var_name must be of type str"
            self._assert_var_defined(assignment.read_arg)

    @unravel_lark_node
    def add_fact(self, fact: AddFact) -> None:
        self._assert_var_terms_defined(fact.term_list, fact.type_list)

    @unravel_lark_node
    def remove_fact(self, fact: RemoveFact) -> None:
        self._assert_var_terms_defined(fact.term_list, fact.type_list)

    @unravel_lark_node
    def query(self, query: Query) -> None:
        self._assert_var_terms_defined(query.term_list, query.type_list)

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:

        # for each relation in the rule body, check if its variable terms are defined
        for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
            if isinstance(relation, Relation):
                self._assert_var_terms_defined(relation.term_list, relation.type_list)
            elif isinstance(relation, IERelation):
                # ie relations have input terms and output terms, check them both
                self._assert_var_terms_defined(relation.input_term_list, relation.input_type_list)
                self._assert_var_terms_defined(relation.output_term_list, relation.output_type_list)
            else:
                raise Exception(f'unexpected relation type: {relation_type}')


## referenced relations

In [ ]:
#| export
class CheckReferencedRelationsExistenceAndArity(InterpreterPass):
    """
    A lark tree semantic check. <br>
    Checks whether each normal relation (that is not an ie relation) reference refers to a defined relation.
    Also checks if the relation reference uses the correct arity.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    def _assert_relation_exists_and_correct_arity(self, relation: Relation) -> None:
        """
        A utility function that checks if a relation exists in the symbol table
        and if the correct arity was used.

        @param relation: the relation that will be checked.
        """

        # get the relation name and the arity that was used by the user
        relation_name = relation.relation_name
        used_arity = len(relation.term_list)

        # check if the relation exists using the symbol table
        if not self.symbol_table.contains_relation(relation_name):
            raise Exception(f'relation "{relation_name}" is not defined')

        # at this point we know the relation exists but we still need to check that the correct arity was used
        # get the correct arity
        relation_schema = self.symbol_table.get_relation_schema(relation_name)
        correct_arity = len(relation_schema)

        # check if that arity that was used is correct
        if used_arity != correct_arity:
            raise Exception(f'relation "{relation_name}" was referenced with an incorrect arity: {used_arity}. The '
                            f'correct arity is: {correct_arity}')

    @unravel_lark_node
    def query(self, query: Query) -> None:
        # a query is defined by a relation reference, so we can simply use the utility function
        self._assert_relation_exists_and_correct_arity(query)

    @unravel_lark_node
    def add_fact(self, fact: AddFact) -> None:
        # a fact is defined by a relation reference, so we can simply use the utility function
        self._assert_relation_exists_and_correct_arity(fact)

    @unravel_lark_node
    def remove_fact(self, fact: RemoveFact) -> None:
        # a fact is defined by a relation reference, so we can simply use the utility function
        self._assert_relation_exists_and_correct_arity(fact)

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:
        """
        A rule is a definition of the relation in the rule head. Therefore the rule head reference does not
        need to be checked.
        The rule body references relations that should already exist. Those will be checked in this method.
        """

        # check that each normal relation in the rule body exists and that the correct arity was used
        for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
            if isinstance(relation, Relation):
                self._assert_relation_exists_and_correct_arity(relation)


In [ ]:
#| export
class CheckReferencedIERelationsExistenceAndArity(VisitorRecursivePass):
    """
    A lark tree semantic check. <br>
    Checks whether each ie relation reference refers to a defined ie function. <br>
    Also checks if the correct input arity and output arity for the ie function were used.

    Currently, an ie relation can only be found in a rule's body, so this is the only place where this
    check will be performed.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:

        # for each ie relation in the rule body, check its existence and arity
        for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
            if isinstance(relation, IERelation):

                # get the name of the ie function, it is the same as the name of the ie relation
                ie_func_name = relation.relation_name

                # assert that the ie function exists
                if not self.symbol_table.contains_ie_function(ie_func_name):
                    raise Exception(f'the information extraction function "{ie_func_name}" does not exist')

                # the ie function exists, gets its data
                ie_func_data = self.symbol_table.get_ie_func_data(ie_func_name)

                # check if the correct input arity was used
                used_input_arity = len(relation.input_term_list)
                correct_input_arity = len(ie_func_data.get_input_types())
                if used_input_arity != correct_input_arity:
                    raise Exception(f'used incorrect input arity for ie function "{ie_func_name}":'
                                    f' {used_input_arity} (should be {correct_input_arity})')

                # check if the correct output arity was used
                used_output_arity = len(relation.output_term_list) + len(relation.input_term_list)
                correct_output_arity = len(ie_func_data.get_output_types(used_output_arity))
                if used_output_arity != correct_output_arity:
                    raise Exception(f'used incorrect output arity for ie function {ie_func_name}:'
                                    f' {used_output_arity} (should be {correct_output_arity})')


## Rule safety

In [ ]:
#| export
class CheckRuleSafety(VisitorRecursivePass):
    """
    Performs semantic checks on rules using a Lark tree to ensure their safety. <br>
    A rule is considered "safe" when it meets certain conditions.


    * [Rule Safety Conditions](#rule-safety-conditions)
        * [Free Variable in Rule Head](#free-variable-in-rule-head)
        * [Bound Free Variable](#bound-free-variable)
    * [Examples](#examples)
    * [Safe Relations](#safe-relations)

    ---

    ### Rule Safety Conditions

    For a rule to be considered safe, the following two conditions must be met:

    ### 1. Free Variable in Rule Head

    Every free variable that appears in the rule head must occur at least once in the body as an output term of a relation.

    #### Examples

    * `parent(X,Y) <- son(X)` is not a safe rule because the free variable `Y` only appears in the rule head.  
    * `parent(X,Z) <- parent(X,Y), parent(Y,Z)` is a safe rule since both `X` and `Z` appear in the rule body.

    ### 2. Bound Free Variable

    A free variable is considered "bound" if it is constrained in a manner that limits the range of values it can take.

    To ensure that every free variable is bound, we must ensure that every relation in the rule body is a safe relation.

    ### Safe Relations

    A safe relation adheres to the following:

    * Its input relation is safe, meaning all its input's free variables are bound. Normal relations are always considered safe as they don't have input relations.  
    * A bound variable is one that exists in the output of a safe relation.

    #### Examples

    * `rel2(X,Y) <- rel1(X,Z), ie1<X>(Y)` is a safe rule as the only input free variable, `X`, exists in the output of the safe relation `rel1(X, Z)`.  
    * `rel2(Y) <- ie1<Z>(Y)` is not safe as the input free variable `Z` does not exist in the output of any safe relation.

    ---


    """

    def __init__(self, **kw: Any) -> None:
        super().__init__()

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:
        head_relation = rule.head_relation
        body_relation_list = rule.body_relation_list
        body_relation_type_list = rule.body_relation_type_list

        # check condition 1:
        # every free variable in the head occurs at least once in the body as an output term of a relation.

        # get the free variables in the rule head
        rule_head_free_vars = get_free_var_names(head_relation.term_list, head_relation.type_list)

        # get the free variables in the rule body that serve as output terms.
        rule_body_output_free_var_sets = [get_output_free_var_names(relation) for relation in body_relation_list]
        rule_body_output_free_vars = set.union(*rule_body_output_free_var_sets)

        # make sure that every free variable in the rule head appears at least once as an output term
        # in the rule body
        bad_rule_head_free_vars = rule_head_free_vars.difference(rule_body_output_free_vars)
        if bad_rule_head_free_vars:
            raise Exception(f'The rule "{rule}" \n'
                            f'is not safe because the following free variables appear in the '
                            'rule head but not as output terms in the rule body:\n'
                            f'{bad_rule_head_free_vars}')

        # check condition 2:
        # every free variable is bound.

        # use a fix point iteration algorithm to find if all the free variables are bound:
        # a. iterate over all of the rule body relations and check if they are safe, meaning all their input
        # free variable terms are bound.
        # b. if a relation is safe, mark its output free variables as bound.
        # c. repeat step 'a' until no new bound free variables are found.

        def get_size_difference(set1: Set, set2: Set) -> int:
            """
            A utility function to be used as the distance function of the fixed point algorithm.

            @return: the size difference of set1 and set2.
            """
            size_difference = abs(len(set1) - len(set2))
            return size_difference

        def get_bound_free_vars(known_bound_free_vars: Set[str]) -> Set[str]:
            """
            a utility function to be used as the step function of the fixed point algorithm.
            this function iterates over all of the rule body relations, checking if each one of them is safe.
            if a rule is found to be safe, this function will mark its output free variables as bound.

            @param known_bound_free_vars: a set of the free variables in the rule that are known to be bound.
            @return: a union of 'known_bound_free_vars' with the bound free variables that were found.
            """

            for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
                # check if all of its input free variable terms of the relation are bound
                input_free_vars = get_input_free_var_names(relation)
                unbound_input_free_vars = input_free_vars.difference(known_bound_free_vars)
                if len(unbound_input_free_vars) == 0:
                    # all input free variables are bound, mark the relation's output free variables as bound
                    output_free_vars = get_output_free_var_names(relation)
                    known_bound_free_vars = known_bound_free_vars.union(output_free_vars)

            return known_bound_free_vars

        # get the bound free variables
        bound_free_vars = fixed_point(start=set(), step=get_bound_free_vars, distance=get_size_difference, thresh=0)

        # get all of the input free variables that were used in the rule body
        rule_body_input_free_var_sets = [get_input_free_var_names(relation)
                                         for relation, relation_type in
                                         zip(body_relation_list, body_relation_type_list)]
        rule_body_input_free_vars = set.union(*rule_body_input_free_var_sets)

        # assert there aren't any unbound free variables
        unbound_free_vars = rule_body_input_free_vars.difference(bound_free_vars)
        if unbound_free_vars:
            # condition 2 check failed, get all of the unbound free variables and pass them in an exception
            raise Exception(f'The rule "{rule}" \n'
                            f'is not safe because the following free variables are not bound:\n'
                            f'{unbound_free_vars}')



## Type check assignment

In [ ]:
#| export
class TypeCheckAssignments(InterpreterPass):
    """
    A lark semantic check <br>
    performs type checking for `Assignments` <br>
    in the current version of lark, this type checking is only required for read assignments.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def read_assignment(self, assignment: ReadAssignment) -> None:

        # get the type of the argument for the read() function
        if assignment.read_arg_type is DataTypes.var_name:
            read_arg_var_name = assignment.read_arg
            read_arg_type = self.symbol_table.get_variable_type(read_arg_var_name)
        else:
            read_arg_type = assignment.read_arg_type

        # if the argument is not of type string, raise and exception
        if read_arg_type is not DataTypes.string:
            raise Exception(f'type checking failed for the read assignment {assignment}\n'
                            f'because the argument type for read() was {read_arg_type} (must be a string)')


## Type check relations

In [ ]:
#| export
class TypeCheckRelations(InterpreterPass):
    """
    A Lark Tree Semantic Check

    ### Assumptions

    This pass operates under the following assumptions. Failure to meet these may lead to incorrect results:

    1. References to relations and IE (Information Extraction) relations, as well as their arity, have been properly checked.
    2. Variable references have been verified.
    3. The pass only processes a single statement as input.

    ### Semantic Checks

    The pass performs the following specific checks:

    #### 1. Typed Relation References

    It verifies if the relation references in the rule are correctly typed.

    #### 2. Typed IE Relations

    It verifies if the IE relations in the rule are correctly typed.

    #### 3. Conflicting Types in Free Variables

    Checks if free variables within rules have conflicting types. This is crucial to ensure that the rules are logically coherent.

    ### Example

    Here is an example that illustrates how a semantic check may fail on the third type of check:

    ```prolog
    new A(str)
    new B(int)
    C(X) <- A(X), B(X)  # Error: X is expected to be both an int and a string.
    ```
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def add_fact(self, fact: AddFact) -> None:
        # a fact is defined by a relation, check if that relation is properly typed
        type_check_passed = check_properly_typed_relation(fact, self.symbol_table)
        if not type_check_passed:
            raise Exception(f'type check failed for fact: "{fact}"')

    @unravel_lark_node
    def remove_fact(self, fact: RemoveFact) -> None:
        # a fact is defined by a relation, check if that relation is properly typed
        type_check_passed = check_properly_typed_relation(fact, self.symbol_table)
        if not type_check_passed:
            raise Exception(f'type check failed for fact: "{fact}"')

    @unravel_lark_node
    def query(self, query: Query) -> None:
        # a query is defined by a relation, check if that relation is properly typed
        type_check_passed = check_properly_typed_relation(query, self.symbol_table)
        if not type_check_passed:
            raise Exception(f'type check failed for query: "{query}"')

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:

        # for each relation in the rule body, check if it is properly typed, raise an exception if it isn't
        for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
            relation_is_properly_typed = check_properly_typed_relation(relation, self.symbol_table)
            if not relation_is_properly_typed:
                raise Exception(f'type check failed for rule "{rule}"\n'
                                f'because the relation "{relation}"\n'
                                f'is not properly typed')

        # check for free variables with conflicting type in the rule, raise an exception if there are any
        _, conflicted_free_vars = type_check_rule_free_vars(rule, self.symbol_table)
        if conflicted_free_vars:
            raise Exception(f'type check failed for rule "{rule}"\n'
                            f'because the following free variables have conflicting types:\n'
                            f'{conflicted_free_vars}')


## Sacvedeclared relations

In [ ]:
#| export
class SaveDeclaredRelationsSchemas(InterpreterPass):
    """
    This pass writes the relation schemas that it finds in relation declarations and rule heads* to the
    symbol table.

    This pass assumes that type checking was already performed on its input.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def relation_declaration(self, relation_decl: RelationDeclaration) -> None:
        self.symbol_table.add_relation_schema(relation_decl.relation_name, relation_decl.type_list, False)

    @no_type_check
    @unravel_lark_node
    def rule(self, rule: Rule) -> None:
        # a rule head relation only contains free variable terms, meaning its schema is defined exclusively by the
        # types of said free variables. a free variable type in a rule can be found using the schemas of relations
        # in the rule body
        # get a mapping from a free variable in this rule to its type
        free_var_to_type, _ = type_check_rule_free_vars(rule, self.symbol_table)

        # get the schema of the rule head relation and add it to the symbol table
        head_relation = rule.head_relation
        term_list = head_relation.term_list
        rule_head_schema = [free_var_to_type[term] for term in term_list]
        self.symbol_table.add_relation_schema(head_relation.relation_name, rule_head_schema, True)


::: {.callout-note}
Note that a rule is a relation declaration of the rule head relation and a definition of its contents
:::

## Resovle var references

In [ ]:
#| export
class ResolveVariablesReferences(InterpreterPass):
    """
    A lark execution pass, <br>
    this pass replaces variable references with their literal values. <br>
    also replaces `DataTypes.var_name` types with the real type of the variable.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def assignment(self, assignment: Assignment) -> None:
        # if the assigned value is a variable, replace it with its literal value
        if assignment.value_type is DataTypes.var_name:
            assigned_value = assignment.value
            assignment.value = self.symbol_table.get_variable_value(assigned_value)
            assignment.value_type = self.symbol_table.get_variable_type(assigned_value)

    @unravel_lark_node
    def read_assignment(self, assignment: ReadAssignment) -> None:
        # if the read() argument is a variable, replace it with its literal value
        if assignment.read_arg_type is DataTypes.var_name:
            read_arg_var_name = assignment.read_arg
            assignment.read_arg = self.symbol_table.get_variable_value(read_arg_var_name)
            assignment.read_arg_type = self.symbol_table.get_variable_type(read_arg_var_name)

    @no_type_check
    def _resolve_var_terms(self, term_list: Sequence[DataTypeMapping.term], type_list: Sequence[DataTypes]) -> None:
        """
        A utility function for resolving variables in term lists
        for each variable term in term_list, replace its value in term_list with its literal value, and
        its DataTypes.var_name type in type_list with its real type
        the changes to the lists are done in-place.

        @param term_list: a list of terms.
        @param type_list: the type of terms in term_list.
        """

        # get the list of terms with resolved variable values
        resolved_var_values_term_list = [
            self.symbol_table.get_variable_value(term) if term_type is DataTypes.var_name
            else term
            for term, term_type in zip(term_list, type_list)]

        # get the list of types with resolved variable types
        resolved_var_types_type_list = [
            self.symbol_table.get_variable_type(term) if term_type is DataTypes.var_name
            else term_type
            for term, term_type in zip(term_list, type_list)]

        # replace the lists with the resolved lists. use slicing to do it in-place.
        term_list[:] = resolved_var_values_term_list
        type_list[:] = resolved_var_types_type_list

    @unravel_lark_node
    def query(self, query: Query) -> None:
        self._resolve_var_terms(query.term_list, query.type_list)

    @unravel_lark_node
    def add_fact(self, fact: AddFact) -> None:
        self._resolve_var_terms(fact.term_list, fact.type_list)

    @unravel_lark_node
    def remove_fact(self, fact: RemoveFact) -> None:
        self._resolve_var_terms(fact.term_list, fact.type_list)

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:
        # resolve the variables of each relation in the rule body relation list
        for relation, relation_type in zip(rule.body_relation_list, rule.body_relation_type_list):
            if isinstance(relation, Relation):
                self._resolve_var_terms(relation.term_list, relation.type_list)
            elif isinstance(relation, IERelation):
                # ie relations have two term lists (input and output), resolve them both
                self._resolve_var_terms(relation.input_term_list, relation.input_type_list)
                self._resolve_var_terms(relation.output_term_list, relation.output_type_list)
            else:
                raise Exception(f'unexpected relation type: {relation_type}')


## Execute assignments

In [ ]:
#| export
class ExecuteAssignments(InterpreterPass):
    """
    A lark execution pass, <br>
    executes assignments by saving variables' values and types in the symbol table <br>
    should be used only after variable references are resolved, meaning the assigned values and read() arguments
    are guaranteed to be literals.
    """

    def __init__(self, symbol_table: SymbolTableBase, **kw: Any) -> None:
        super().__init__()
        self.symbol_table = symbol_table

    @unravel_lark_node
    def assignment(self, assignment: Assignment) -> None:
        # perform the assignment by saving the variable attributes in the symbol table
        self.symbol_table.set_var_value_and_type(assignment.var_name, assignment.value, assignment.value_type)

    @unravel_lark_node
    def read_assignment(self, assignment: ReadAssignment) -> None:
        # try to read the file and get its content as a single string. this string is the assigned value.
        try:
            assigned_value = Path(assignment.read_arg).read_text()
        except Exception:
            raise Exception(f'could not open file "{assignment.read_arg}"')

        # perform the assignment by saving the variable attributes in the symbol table
        # note that since this is a read assignment, the type of the variable will always be a string
        self.symbol_table.set_var_value_and_type(assignment.var_name, assigned_value, DataTypes.string)


## Add statement to netxparse graph

No clue why we need this

In [ ]:
#| export

#TODO agg - rewrite this to not exist? and if it exist use a naive netx graph
class AddStatementsToNetxParseGraph(InterpreterPass):
    """
    A lark execution pass. <br>
    This pass adds each statement in the input parse tree to the parse graph. <br>
    This pass is made to work with execution.naive_execution as the execution function and
    `term_graph.NetxStateGraph` as the parse graph.

    Each statement in the parse graph will be a child of the parse graph's root.

    Each statement in the parse graph will have a type attribute that contains the statement's name in the
    spannerlog grammar.

    Some nodes in the parse graph will contain a value attribute that would contain a relation that describes
    that statement.
    e.g. a `add_fact` node would have a value which is a `structured_nodes.AddFact` instance
    (which inherits from `structured_nodes.Relation`) that describes the fact that will be added.

    Some statements are more complex and will be described by more than a single node, e.g. a rule node.
    The reason for this is that we want a single netx node to not contain more than one Relation
    (or `IERelation`) instance. This will make the parse graph a "graph of relation nodes", allowing
    for flexibility for optimization in the future.
    """

    def __init__(self, parse_graph, **kw: Any) -> None:
        super().__init__()
        self.parse_graph = parse_graph

    def _add_statement_to_parse_graph(self, statement_type: ParseNodeType, statement_value: Any) -> None:
        """
        A utility function that adds a statement to the parse graph, meaning it adds a node that
        represents the statement to the parse graph, then attach the node to the parse graph's root.
        Should only be used for simple statements (i.e. can be described by a single node).

        @param statement_type: the type of the statement, (e.g. add_fact). should be the same as the statement's
                               name in the grammar. Will be set as the node's type attribute.
        @param statement_value: will be set as the value attribute of the node.
        """
        new_statement_node = self.parse_graph.add_node(type=statement_type, value=statement_value)
        self.parse_graph.add_edge(self.parse_graph.get_root_id(), new_statement_node)

    @unravel_lark_node
    def add_fact(self, fact: AddFact) -> None:
        self._add_statement_to_parse_graph(ParseNodeType.ADD_FACT, fact)

    @unravel_lark_node
    def remove_fact(self, fact: RemoveFact) -> None:
        self._add_statement_to_parse_graph(ParseNodeType.REMOVE_FACT, fact)

    @unravel_lark_node
    def query(self, query: Query) -> None:
        self._add_statement_to_parse_graph(ParseNodeType.QUERY, query)

    @unravel_lark_node
    def relation_declaration(self, relation_decl: RelationDeclaration) -> None:
        self._add_statement_to_parse_graph(ParseNodeType.RELATION_DECLARATION, relation_decl)

    @unravel_lark_node
    def rule(self, rule: Rule) -> None:
        self._add_statement_to_parse_graph(ParseNodeType.RULE, rule)
